# Clara Train SDK Hyper Parameter Tuning Using AutoML

By the end of this notebook you should be able to:
1. Run AutoML and use it as a scheduler for hyperparameter tuning to search:
    1. Network architecture.
    2. loss functions 
    3. Optimizers
    4. Transformations
2. Run AutoML with reinforcement learning to search float parameters such as:
    1. Learning rate
    2. Transformation probabilities and ranges


## Prerequisites
- Familiar with Clara train main concepts. See [Getting Started Notebook](../GettingStarted/GettingStarted.ipynb)
- Nvidia GPU with 8Gb of memory  
- Have multiple GPUs is preferred   


### Resources
It might be helpful to watch the GTC Digital 2020 presentation on Clara Train SDK 
- [S22563](https://developer.nvidia.com/gtc/2020/video/S22563)
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


## Dataset 
This notebook uses a sample dataset (ie. a single image volume of the spleen dataset) provided in the package to train a small neural network for a few epochs. 
This single file is duplicated 32 times for the training set and 9 times for validation to mimic the full spleen data set. 


# Lets get started
It might be helpful to check the available NVIDIA GPU resources in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi

This cell we define the root path for AutoML

In [ ]:
MMAR_ROOT="/claraDevDay/AutoML/"
print ("setting MMAR_ROOT=",MMAR_ROOT)
%ls $MMAR_ROOT


This cell defines some helper functions that we will use throughout the notebook

In [ ]:
!chmod 777 $MMAR_ROOT/commands/*
def runAutoML(configPath):
    %cd $MMAR_ROOT/commands
    ! ./automl.sh $configPath
def printFile(filePath,lnSt,lnOffset):
    print ("showing ",str(lnOffset)," lines from file ",filePath, "starting at line",str(lnSt))
    lnOffset=lnSt+lnOffset
    !< $filePath head -n "$lnOffset" | tail -n +"$lnSt"


 # AutoML Main Components
- [automl.sh](./commands/automl.sh) is where we configure the number of workers and which gpus to use. 
In this notebook we will use very small neural networks that will consume <2GB of GPU memory. 
We are setting the number of workers to 8 and they will all use GPU 0 by specifying 
`workers=0:0:0:0:0:0:0:0`. 
If you have multiple GPU and would like to have more workers you could change this to:
    - `workers=0:0:0:0:1:1:1:1` this will let workers 0-3 use GPU 0 while workers 4-7 will use GPU 1
    - `workers=0:1:2:3` this will let 4 workers each use a GPU
    - `workers=0,1:2,3` this will let 2 workers each use a 2 GPUs

lets check out the contents of automl.sh

In [ ]:
configFile=MMAR_ROOT+"/commands/automl.sh"
printFile(configFile,0,30)

- [config_automl.json](./config/config_automl.json) is where any custom controls would be defined 
along with maximum number of MMARs to search and the number of MMARs to keep

In [ ]:
configFile=MMAR_ROOT+"/config/config_automl.json"
printFile(configFile,0,30)


- [automl_train_round.sh](./commands/automl_train_round.sh) is the script which will be triggered for each job  


In [ ]:
configFile=MMAR_ROOT+"/commands/automl_train_round.sh"
printFile(configFile,30,30)


# Using Auto ML for Hyper Parameter Enum Search

You can use AutoML as a scheduler to run multiple configurations using the Enum options. 


### Example 1: Network Parameter Search
In this example we are using the Enum option to search different network
 architecture arguments for the SegResnet neural network using [trn_autoML_Enum.json](./config/trn_autoML_Enum.json):
- Blocks_down, blocks_up
- Init_filters
- Final_activation


In [ ]:
# lets see how to define this in the config
configFile=MMAR_ROOT+"/config/trn_autoML_Enum.json"
printFile(configFile,46,28)
# we also seach different lr policies 
printFile(configFile,32,13)


let  us run it a see the generated configurations


In [ ]:
runAutoML("trn_autoML_Enum")


AutoML now has created a sub-folder under automl/  and created 8 folders for all combinations of our experiments. 
Let us run the cell below and see the folders created  

In [ ]:
! ls -la $MMAR_ROOT/automl/trn_autoML_Enum

Let us now examine each config generated from the a couple of experiments

In [ ]:
exp_name="trn_autoML_Enum"
for i in range(1,3):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,32,5)
for i in range(1,6,2):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,38,12)



### Example 2: Loss and optimizer Search   
In this Example we are using the Enum option to search different losses 
 using [trn_autoML_Loss_Optimizer.json](./config/trn_autoML_Loss_Optimizer.json). 
 In order to do this we 
- use first level search to create an binary Enum tag set to `[true, false]` 
- apply the first tag to loss
- apply the second tag to optimizer  


In [ ]:
# lets see how to define this in the config
configFile=MMAR_ROOT+"/config/trn_autoML_Loss_Optimizer.json"
# top level binary Enum
printFile(configFile,8,14)
# apply to loss
printFile(configFile,25,5)
# apply to optimizer 
printFile(configFile,32,5)


In [ ]:
# lets run it a see configurations generated 
runAutoML("trn_autoML_Loss_Optimizer")

Lets run cell below and see the folders created

In [ ]:
! ls -la $MMAR_ROOT/automl/trn_autoML_Loss_Optimizer

Lets now examine each config generated from the a couple of experiments

In [ ]:
exp_name="trn_autoML_Loss_Optimizer"
for i in [1,2]:
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,9,16)
for i in [1,3]:
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,25,12)


# Using Auto ML for float parameter using reinforcement learning (RL)
So far we have used only the Enum option of AutoML. But what if we have a float value? 
We now will set a range in the search space and use reinforcement learning (RL). 
    

### Example 3: Learning Rate and Learning Policy (RL) Search
In this Example we will use [trn_autoML_LR_LP.json](./config/trn_autoML_LR_LP.json). 
 In order to do this we 
- Use first level search learning rate 
- Search for learning policy   

Note that this will only use 1 worker since we will run a model wait for result then trigger another.

In [ ]:
# lets see how to define this in the config
configFile=MMAR_ROOT+"/config/trn_autoML_LR_LP.json"
# top level search for learning rate
printFile(configFile,8,10)
# search for learning policy 
printFile(configFile,40,10)


In [ ]:
# lets run it a see configurations generated 
runAutoML("trn_autoML_LR_LP")

Lets run cell below and see the folders created

In [ ]:
! ls -la $MMAR_ROOT/automl/trn_autoML_LR_LP

Lets now examine each config generated from the a couple of experiments

In [ ]:
exp_name="trn_autoML_LR_LP"
for i in range(2,5):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,5,2)
for i in range(2,6,1):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,32,4)



### Example 4: Transformation Search   
In this Example we are using Enum option to search different transforms 
using [trn_autoML_Transform.json](./config/trn_autoML_Transform.json). 
Here, we will use Enum to enable/disable `AddGaussianNoise` transform and also use top level Enum 
to link 2 transforms to be enabled/disabled together. 
In order to do this we 
- Use first level search to create an binary Enum tag set to `[true, false]` and 
tag it with args `myTransformDisable` 
- Apply this tag `myTransformDisable` to transforms `RandomSpatialFlip`
- Apply this tag `myTransformDisable` to transforms `ScaleShiftIntensity`
- Add `search` section to `AddGaussianNoise` with a `"args": ["@disabled"]` and `"targets": [[true],[false]]`

In [ ]:
# lets see how to define this in the config
configFile=MMAR_ROOT+"/config/trn_autoML_Transform.json"
# top level binary Enum
printFile(configFile,8,14)
# apply to 1st transform
printFile(configFile,115,10)
# apply to 2nd transform 
printFile(configFile,126,10)
# Add separate search for AddGaussianNoise transform 
printFile(configFile,140,12)


In [ ]:
# lets run it a see configurations generated 
runAutoML("trn_autoML_Transform")

Let us run cell below and see the folders created

In [ ]:
! ls -la $MMAR_ROOT/automl/trn_autoML_Transform

Let us now examine each config generated from the a couple of experiments

In [ ]:
exp_name="trn_autoML_Transform"
for i in range(1,5):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,125,16)
for i in range(1,4,2):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,153,7)
    

### Example 5: Transform Reinforcement Learning (RL) Search with linking   
In this Example we are using RL option to search float parameter as well as linking these parameters between 2 transforms 
 using [trn_autoML_TransformProb.json](./config/trn_autoML_TransformProb.json). 
 In order to do this we 
- Set a search in the first transformation `ScaleIntensityOscillation`
- Give this transform `ScaleIntensityOscillation` an alias like `myProb` 
- Apply this alias to the second transform `AddGaussianNoise` using the `apply` field 


In [ ]:
# lets see how to define this in the config
configFile=MMAR_ROOT+"/config/trn_autoML_TransformProb.json"
# Search first transformation ScaleIntensityOscillation and give a tag
printFile(configFile,120,15)
# apply to AddGaussianNoise transformation 
printFile(configFile,134,9)


In [ ]:
# lets run it a see configurations generated 
runAutoML("trn_autoML_TransformProb")

Let us run cell below and see the folders created

In [ ]:
! ls -la $MMAR_ROOT/automl/trn_autoML_TransformProb

Let us now examine each config generated from the a couple of experiments

In [ ]:
exp_name="trn_autoML_TransformProb"
for i in range(2,7):
    configFile=MMAR_ROOT+"/automl/"+exp_name+"/W"+str(i)+"_1_J"+str(i)+"/config/config_train.json"
    printFile(configFile,144,16)
    

# BYO AutoML logic 
As you could BYOC for regular components in train configuration, 
you could also bring your own AutoML logic. 
To do this, you simply have to follow the programmers guide and implement the APIs. 
As an example we are providing a simple controller [myAutoMLController.py](./BYOC/myAutoMLController.py). 
you should then point to in the [config_automl.json](./config/config_automl.json). 
Unfortunately you can't overwrite that name so we have provided it in [config_automl_.json](./config/config_automl_.json).


# Exercise
1. You can use BYO `ScaleIntensityRange` transformation created in Example 1 in [BYOC notebook](../GettingStarted/BYOC.ipynb)
to do a search on BYO Transform. _Hint: you must use ref in the validation transforms to use the same values as in the training transforms_ 
2. You should change different parameters of automl and do more search  
3. You can now change / rename the config_automl.json to point to the custom controller and try using it.
